# 1. **Imports**

In [ ]:
from google.colab import files
files.upload()


{}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!pip install kagglehub


In [ ]:
import kagglehub

path = kagglehub.dataset_download("harshitpathak18/hand-sign-dataset")
print("Dataset path:", path)


Using Colab cache for faster access to the 'hand-sign-dataset' dataset.
Dataset path: /kaggle/input/hand-sign-dataset


In [ ]:
!pip install -q kagglehub tensorflow opencv-python-headless

import os
import random
import numpy as np
import cv2
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import kagglehub

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

IMG_SIZE = 64
MAX_IMAGES_PER_CLASS = 1500  # increase samples per class if available
BATCH_SIZE = 64
EPOCHS = 15


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.19.0


# 2. Dataset Loading

In [ ]:
# Download latest version of the dataset
dataset_path = kagglehub.dataset_download("harshitpathak18/hand-sign-dataset")
print("Dataset path:", dataset_path)

# If dataset has a 'train' folder, use it, otherwise use root
train_path = os.path.join(dataset_path, "train") if os.path.exists(os.path.join(dataset_path, "train")) else dataset_path
print("Using images from:", train_path)


Using Colab cache for faster access to the 'hand-sign-dataset' dataset.
Dataset path: /kaggle/input/hand-sign-dataset
Using images from: /kaggle/input/hand-sign-dataset


# 3. Model Architecture

In [ ]:
import os # Added import statement for os module

def load_images_from_folder(base_path, img_size=64, max_per_class=1500):
    images = []
    labels = []

    print("Loading images...")
    for folder_name in sorted(os.listdir(base_path)):
        folder_path = os.path.join(base_path, folder_name)
        if not os.path.isdir(folder_path):
            continue

        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(image_files)
        selected_files = image_files[:max_per_class]

        count = 0
        for filename in selected_files:
            img_path = os.path.join(folder_path, filename)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                if img is None:
                    continue

                # Optional: crop/center if needed, then resize
                img = cv2.resize(img, (img_size, img_size))
                images.append(img)
                labels.append(folder_name)
                count += 1
            except Exception as e:
                print(f"Skipping {img_path}: {e}")
        print(f"Class {folder_name}: loaded {count} images")

    print(f"Total images loaded: {len(images)}")
    return np.array(images, dtype='float32'), np.array(labels)

# Adjust train_path to point to the 'data' subfolder if it exists
# This assumes the images are nested under a 'data' directory within the downloaded dataset.
if os.path.exists(os.path.join(train_path, "data")):
    train_path = os.path.join(train_path, "data")
    print(f"Adjusted train_path to: {train_path}")

# Check for another 'data' subfolder, if the structure is deeply nested like data/data/0...
if os.path.exists(os.path.join(train_path, "data")):
    train_path = os.path.join(train_path, "data")
    print(f"Further adjusted train_path to: {train_path}")

X_raw, y_raw = load_images_from_folder(train_path, IMG_SIZE, MAX_IMAGES_PER_CLASS)

# Normalize and shuffle
X = X_raw / 255.0
X, y = shuffle(X, y_raw, random_state=SEED)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = to_categorical(y_encoded)

# Save label encoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

num_classes = len(le.classes_)
print("Classes:", le.classes_)
print("Number of classes:", num_classes)

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y_onehot, test_size=0.2, random_state=SEED, stratify=y_onehot
)
X_train.shape, X_val.shape


# **4.Training**

In [ ]:
def build_model(input_shape=(64, 64, 3), num_classes=10):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),

        Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=num_classes)
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 27)             │         3,483 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,227,931 (8.50 MB)

 Trainable params: 2,227,483 (8.50 MB)

 Non-trainable params: 448 (1.75 KB)

# **5.Save Model**

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

model.save("asl_cnn_model.h5")
print("Model saved as asl_cnn_model.h5")


Epoch 1/15
169/169 ━━━━━━━━━━━━━━━━━━━━ 144s 834ms/step - accuracy: 0.3066 - loss: 2.8417 - val_accuracy: 0.0534 - val_loss: 6.9629 - learning_rate: 0.0010
Epoch 2/15
169/169 ━━━━━━━━━━━━━━━━━━━━ 141s 830ms/step - accuracy: 0.6993 - loss: 0.9542 - val_accuracy: 0.3814 - val_loss: 2.9889 - learning_rate: 0.0010
Epoch 3/15
169/169 ━━━━━━━━━━━━━━━━━━━━ 137s 811ms/step - accuracy: 0.7964 - loss: 0.6549 - val_accuracy: 0.8879 - val_loss: 0.3073 - learning_rate: 0.0010
Epoch 4/15
169/169 ━━━━━━━━━━━━━━━━━━━━ 144s 824ms/step - accuracy: 0.8194 - loss: 0.5673 - val_accuracy: 0.9948 - val_loss: 0.0514 - learning_rate: 0.0010
Epoch 5/15
169/169 ━━━━━━━━━━━━━━━━━━━━ 138s 817ms/step - accuracy: 0.8461 - loss: 0.4773 - val_accuracy: 0.9581 - val_loss: 0.2269 - learning_rate: 0.0010
Epoch 6/15
169/169 ━━━━━━━━━━━━━━━━━━━━ 136s 807ms/step - accuracy: 0.8564 - loss: 0.4460 - val_accuracy: 0.9777 - val_loss: 0.1338 - learning_rate: 0.0010
Epoch 7/15
169/169 ━━━━━━━━━━━━━━━━━━━━ 142s 808ms/step - accura

Model saved as asl_cnn_model.h5
